In [32]:
from __future__ import print_function
import cv2
import numpy as np

In [36]:
MAX_FEATURES = 500
GOOD_MATCH_PERCENT = 0.15

In [37]:
def align_images(im1, im2):
    # Convert images to grayscale
    im1Gray = cv2.cvtColor(im1, cv2.COLOR_BGR2GRAY)
    im2Gray = cv2.cvtColor(im2, cv2.COLOR_BGR2GRAY)
   # Detect ORB features and compute descriptors.
    orb = cv2.ORB_create(MAX_FEATURES)
    keypoints1, descriptors1 = orb.detectAndCompute(im1Gray, None)
    keypoints2, descriptors2 = orb.detectAndCompute(im2Gray, None)
  # Match features.
    bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
    matches = bf.match(descriptors1,descriptors2)
    matches = sorted(matches, key = lambda x:x.distance)
  # Sort matches by score
    matches.sort(key=lambda x: x.distance, reverse=False)
  # Draw top matches
    imMatches = cv2.drawMatches(im1, keypoints1, im2, keypoints2, matches,None,flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)
    cv2.imwrite("matches.jpg", imMatches)
  # Extract location of good matches
    points1 = np.zeros((len(matches), 2), dtype=np.float32)
    points2 = np.zeros((len(matches), 2), dtype=np.float32) 
    for i, match in enumerate(matches):
        points1[i, :] = keypoints1[match.queryIdx].pt
        points2[i, :] = keypoints2[match.trainIdx].pt
  # Find homography
    h, mask = cv2.findHomography(points1, points2, cv2.RANSAC)

  # Use homography
    height, width, channels = im2.shape
    im1Reg = cv2.warpPerspective(im1, h, (width, height))
    return im1Reg, h

In [39]:
if __name__ == '__main__':
    
  # Read reference image
    refFilename = "sweatpatch.JPG"
    print("Reading reference image : ", refFilename)
    imReference = cv2.imread(refFilename, cv2.IMREAD_COLOR)
    
  # Read image to be aligned
    imFilename = "Image from iOS.jpg"
    print("Reading image to align : ", imFilename);
    im = cv2.imread(imFilename, cv2.IMREAD_COLOR)
    print("Aligning images ...")

  # Registered image will be resotred in imReg.
  # The estimated homography will be stored in h.
    imReg, h = align_images(im, imReference)

  # Write aligned image to disk.
    outFilename = "aligned.jpg"
    print("Saving aligned image : ", outFilename);
    cv2.imwrite(outFilename, imReg)
    
  # Print estimated homography
    print("Estimated homography : \n",  h)


Reading reference image :  sweatpatch.JPG
Reading image to align :  Image from iOS.jpg
Aligning images ...
Saving aligned image :  aligned.jpg
Estimated homography : 
 [[ 6.21385014e-02  9.26307530e-01 -9.84884278e+02]
 [-8.97188951e-01  5.06296654e-03  2.06108642e+03]
 [ 5.48954534e-05  1.20982336e-05  1.00000000e+00]]
